# English Score

Просмотр фильмов на оригинальном языке - это популярный и действенный метод прокачаться при изучении иностранных языков. Важно выбрать фильм, который подходит студенту по уровню сложности, т.ч. студент понимал 50 - 70 % диалогов. Чтобы выполнить это условие, преподаватель должен посмотреть фильм и решить, какому уровню он соответствует. Однако это требует больших временных затрат.

#### Задача проекта:

Разработать ML решение для автоматического определения уровня сложности англоязычных фильмов.

#### Вводные данные:
1. Файл excel с указанием фильма и разметкой уровня языковой сложности.
2. Субтитры к указанным в файле Excel.
3. Папки с субтитрами, распределенные по разным уровням сложности.

## 1. Загрузка данных

Для начала загрузим все необходимые библиотеки и имеющиеся данные, на основании которых дулее будет происходить обучение модели.

pip install pysrt

pip install pymorphy2

pip uninstall nltk

pip install -U nltk

In [3]:
import pandas as pd
from pathlib import Path
import pysrt
import codecs
from sklearn.utils import shuffle
import nltk
import numpy as np
from nltk.stem.porter import *
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool, cv
from sklearn import metrics
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yulia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

- Загрузка excel с уровнями к каждому фильму

Загрузим файл `movies_labels.xlsx` и посмотрим на его содержимое.

In [4]:
df_xlsx = pd.read_excel('movies_labels.xlsx')
df_xlsx

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+
...,...,...,...
236,236,Matilda(2022),C1
237,237,Bullet train,B1
238,238,Thor: love and thunder,B2
239,239,Lightyear,B2


Данный файл содержит в себе название фильма и уровень сложности языка к нему. Поскльку файлы с субтитрами содержат в наименовании названия фильмов, мы сможем по дынной информации соотнести каждому субтитру уровень из файла excel.

Поскольку одному фильму может быть присвоено сразу несколько уровней сложности, выберем для дальнейшего обучения самый первый указанный уровень сложности.

In [5]:
df_xlsx['Level'] = df_xlsx['Level'].str[0:2]
df_xlsx

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2
...,...,...,...
236,236,Matilda(2022),C1
237,237,Bullet train,B1
238,238,Thor: love and thunder,B2
239,239,Lightyear,B2


- Загрузка списка .srt файлов к файлу excel

Загрузим файлы с субтитрами для фильмов из файла excel. Для этого сначала переберем полные пути к этим файлам.

In [6]:
p = Path("C:/Users/yulia/OneDrive/Документы/С рабочего стола/Проекты/Workshop_2/English_scores/Subtitles_all/Subtitles")
list_str = []

for x in p.rglob("*"):
    list_str.append(str(x))
df_srt = pd.DataFrame(list_str, columns = ['Path'])
df_srt

,Path
0,C:\Users\yulia\OneDrive\Документы\С рабочего с...
1,C:\Users\yulia\OneDrive\Документы\С рабочего с...
2,C:\Users\yulia\OneDrive\Документы\С рабочего с...
3,C:\Users\yulia\OneDrive\Документы\С рабочего с...
4,C:\Users\yulia\OneDrive\Документы\С рабочего с...
...,...
111,C:\Users\yulia\OneDrive\Документы\С рабочего с...
112,C:\Users\yulia\OneDrive\Документы\С рабочего с...
113,C:\Users\yulia\OneDrive\Документы\С рабочего с...
114,C:\Users\yulia\OneDrive\Документы\С рабочего с...


Добавим колонку `Movie` и обработаем в нее названия фильмов из папки с субтитрами, чтобы далее по ней можно было подтянуть для каждого субтитра уровень из первой таблицы.

In [7]:
def add_column_movie(value):
    val = value['Path']
    val = val.replace('C:\\Users\\yulia\\OneDrive\\Документы\\С рабочего стола\\Проекты\\Workshop_2\\English_scores\\Subtitles_all\\Subtitles\\', '')     
    val = val.replace('.srt', '')     
    return val

df_srt['Movie'] = df_srt.apply(add_column_movie, axis=1)
df_srt

,Path,Movie
0,C:\Users\yulia\OneDrive\Документы\С рабочего с...,.DS_Store
1,C:\Users\yulia\OneDrive\Документы\С рабочего с...,10_Cloverfield_lane(2016)
2,C:\Users\yulia\OneDrive\Документы\С рабочего с...,10_things_I_hate_about_you(1999)
3,C:\Users\yulia\OneDrive\Документы\С рабочего с...,Aladdin(1992)
4,C:\Users\yulia\OneDrive\Документы\С рабочего с...,All_dogs_go_to_heaven(1989)
...,...,...
111,C:\Users\yulia\OneDrive\Документы\С рабочего с...,Warm_bodies(2013)
112,C:\Users\yulia\OneDrive\Документы\С рабочего с...,Westworld_scenes_of_Dr_Robert_Ford
113,C:\Users\yulia\OneDrive\Документы\С рабочего с...,We_are_the_Millers(2013)
114,C:\Users\yulia\OneDrive\Документы\С рабочего с...,While_You_Were_Sleeping(1995)


- Добавление в список excel директорий их субтитов

Получим единую таблицу, содержащую информацию об уровне фильма и местоположнии субтитра.

In [8]:
df_fin = df_xlsx.merge(df_srt, left_on='Movie', right_on='Movie')
df_fin

,id,Movie,Level,Path
0,0,10_Cloverfield_lane(2016),B1,C:\Users\yulia\OneDrive\Документы\С рабочего с...
1,1,10_things_I_hate_about_you(1999),B1,C:\Users\yulia\OneDrive\Документы\С рабочего с...
2,2,A_knights_tale(2001),B2,C:\Users\yulia\OneDrive\Документы\С рабочего с...
3,3,A_star_is_born(2018),B2,C:\Users\yulia\OneDrive\Документы\С рабочего с...
4,4,Aladdin(1992),A2,C:\Users\yulia\OneDrive\Документы\С рабочего с...
...,...,...,...,...
105,107,Venom(2018),B2,C:\Users\yulia\OneDrive\Документы\С рабочего с...
106,108,Warm_bodies(2013),B1,C:\Users\yulia\OneDrive\Документы\С рабочего с...
107,109,We_are_the_Millers(2013),B1,C:\Users\yulia\OneDrive\Документы\С рабочего с...
108,110,While_You_Were_Sleeping(1995),B1,C:\Users\yulia\OneDrive\Документы\С рабочего с...


Теперь мы можем произвести непосредственную загрузу содержимого файлов с субтритрами по их полному пути хранения. Для перебора всей таблицы сделаем отдельную функцию.

Поскольку далее нам понадобятся только колонки `Srt` с субтитрами и `Level` с уровнем английского, выберем только их. Выведем получившийся результат на экран.

In [9]:
def add_column_srt(value):
    file = str(value['Path'].replace('\\', '/'))
    text = ''    
    subs = pysrt.open(file, encoding='iso-8859-1')
    srt = ''
    for sub in subs:
        srt = srt + ' ' + sub.text
     
    return srt.replace('\n', ' ')

df_fin['Srt'] = df_fin.apply(add_column_srt, axis=1)
df_fin = df_fin[['Srt','Level']]
df_fin

,Srt,Level
0,"<font color=""#ffff80""><b>Fixed & Synced by bo...",B1
1,"Hey! I'll be right with you. So, Cameron. Her...",B1
2,Resync: Xenzai[NEF] RETAIL Should we help him...,B2
3,"- <i><font color=""#ffffff""> Synced and correc...",B2
4,"<i>Oh, I come from a land From a faraway plac...",A2
...,...,...
105,"<i>Life Foundation Control, this is LF1.</i> ...",B2
106,<i>What am I doing</i> <i>with my life?</i> <...,B1
107,"<i>Oh, my God...</i> <i>...it's full-on doubl...",B1
108,"LUCY: <i>Okay, there are two things that</i> ...",B1


- Загрузка субтитров из папок по уровням английского языка

Поскольку у нас также имеются папки по уровням английского языка, содержащие субтитры, загрузим их в отдельную таблицу с аналогичной структурой.

In [10]:
def add_column_level(value):
    val = value['Path'].split('\\')
    return val[10]

In [11]:
list_str = []
level_list = ['A2', 'B1', 'B2', 'C1']

for level in level_list:
    p = Path("C:/Users/yulia/OneDrive/Документы/С рабочего стола/Проекты/Workshop_2/English_scores/Subtitles_all"+"/"+level)
    for x in p.rglob("*"):
        list_str.append(str(x))
        
df_add = pd.DataFrame(list_str, columns = ['Path'])

df_add['Srt'] = df_add.apply(add_column_srt, axis=1)
df_add['Level'] = df_add.apply(add_column_level, axis=1)
df_add = df_add[['Srt', 'Level']]
df_add

,Srt,Level
0,( bugs chittering ) ( brakes squeak ) - ( eng...,A2
1,- ( birds chirping ) - ( bugs chittering ) Bo...,A2
2,( thunder rumbling ) Merle: That's right. You...,A2
3,( birds chirping ) - What? - Nothing. It's no...,A2
4,"- ( walkie-talkie squawks ) - Rick: Morgan, I...",A2
...,...,...
158,I lost Ava her company. I assume my deal with...,C1
159,Previously on <i>Suits...</i> It's going up o...,C1
160,"I get Ava Hessington acquitted, Darby backs m...",C1
161,Previously on <i>Suits...</i> I'm bonding wit...,C1


- Соединяем 2 таблицы

Теперь мы можем соединить две таблицы для получения большего пула данных для обучения нашей модели.

In [12]:
df_full = pd.concat([df_fin, df_add])
df_full

,Srt,Level
0,"<font color=""#ffff80""><b>Fixed & Synced by bo...",B1
1,"Hey! I'll be right with you. So, Cameron. Her...",B1
2,Resync: Xenzai[NEF] RETAIL Should we help him...,B2
3,"- <i><font color=""#ffffff""> Synced and correc...",B2
4,"<i>Oh, I come from a land From a faraway plac...",A2
...,...,...
158,I lost Ava her company. I assume my deal with...,C1
159,Previously on <i>Suits...</i> It's going up o...,C1
160,"I get Ava Hessington acquitted, Darby backs m...",C1
161,Previously on <i>Suits...</i> I'm bonding wit...,C1


Удалим все пустые значения, перемешаем данные и пересчитаем индексы.

Также первоначальные данные содержат следующие уровни английского языка: `A2`, `B1`, `B2`, `C1`. Для удобства классифицируем уровень `A2` как `A1-A2`, а `C1` как `C1-C2`. 

In [13]:
def replace_level(value):
    val = value['Level']
    val = val.replace('A2', 'A1-A2')
    val = val.replace('C1', 'C1-C2')
    return val

In [14]:
df = shuffle(df_full[df_full['Srt']!='']).reset_index(drop=True)
df['Level'] = df.apply(replace_level, axis=1)
df

,Srt,Level
0,<font color=#FFFF00><b><i>(HARVEY READING)</i...,B2
1,Cinderella You're as lovely as your name Cind...,B1
2,[CROWD CHEERING IN DISTANCE] LElGH ANNE: Ther...,B1
3,You'd better get married soon. You're startin...,A1-A2
4,<font color=#32a615><b>WordsKill</b></font> <...,B2
...,...,...
252,<font color=#FFFF00><b><i>It's a no. I though...,B2
253,America is an irradiated wasteIand. Within it...,A1-A2
254,"Now, where to begin? The first thing you need...",B2
255,"<i>Oh, I come from a land From a faraway plac...",A1-A2


## 2. Подготовка данных к обучению

- Удаление лишних символов

В рамках проведения анализов субтитров были выявлены символы и строки, не несущие в себе смысловой нагрузки и засоряющие данные для обучения модели. Это какие-либо дополнительные сведения о создателе субтитров, веб-сайты, предоставившие субтитры бесплатно, а также технические символы.

После обработки выведем наш датафрейм и посмотрим, как он изменился.

In [15]:
def replace_str(value):
    val = value['Srt']
    val = val.replace('ahmedhamdy90', ' ')
    val = val.replace('subscene.com @ahmedhamdy2121', ' ')
    val = val.replace('https://twitter.com/kaboomskull', ' ')
    val = val.replace('shokomovies.blogspot.com', ' ')
    val = val.replace('UNRATED.720p.BluRay.x264-REFiNED.English Upload to subscene.com by ViSTAâ\x84¢-HDVN', ' ')
    val = val.replace('www.YIFY-TORRENTS.com', ' ')
    val = val.replace('www.ncicap.org', ' ')
    val = val.replace('truecostmovie.com', ' ')
    val = val.replace('Created and Encoded by --  Bokutox -- of  www.YIFY-TORRENTS.com. The Best 720p/1080p/3d movies with the lowest file size on the internet.', ' ')
    val = val.replace('www.facemash.com', ' ')
    val = val.replace('www.OpenSubtitles.org', ' ')
    val = val.replace('www.titlovi.com', ' ')
    val = val.replace('www.AllSubs.org', ' ')
    val = val.replace('www.flixify.app', ' ')
    val = val.replace('truecostmovie.com', ' ')
    val = val.replace('teenybikini.com', ' ')

    val = val.lower()

    val = val.replace('font', ' ')
    val = val.replace('color', ' ')
    val = val.replace('ffff', ' ')
    val = val.replace('<b>', ' ')
    val = val.replace('</b>', ' ')
    val = val.replace('<i>', ' ')
    val = val.replace('</i>', ' ')
    val = val.replace('font color', ' ')
    val = val.replace('</font>', ' ')
    val = val.replace('â\x99ª', ' ')
    val = val.replace('d900d9', ' ')
    val = val.replace('âª', ' ')
    
    return val

df['Srt'] = df.apply(replace_str, axis=1)
df

,Srt,Level
0,< =# 00> (harvey reading) </ > < =# 00>...,B2
1,cinderella you're as lovely as your name cind...,B1
2,[crowd cheering in distance] lelgh anne: ther...,B1
3,you'd better get married soon. you're startin...,A1-A2
4,< =#32a615> wordskill </ > < =#ff0000> ed...,B2
...,...,...
252,< =# 00> it's a no. i thought you liked th...,B2
253,america is an irradiated wasteiand. within it...,A1-A2
254,"now, where to begin? the first thing you need...",B2
255,"oh, i come from a land from a faraway place ...",A1-A2


- Лемматизация

Лемматизация – это алгоритмический процесс нахождения леммы слова в зависимости от его значения. Лемматизация обычно относится к морфологическому анализу слов, целью которого является удаление флективных окончаний. Это помогает в возвращении базовой или словарной формы слова, которое известно как лемма. Метод лемматизации NLTK основан на встроенной морф-функции WorldNet. Предварительная обработка текста включает в себя как основы, так и лемматизации.

Проведем лемматизацию для наших субтитров, а также избавимся от лишних символов.

In [16]:
patterns = "[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_en = stopwords.words("english")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_en:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

df['Srt'] = df['Srt'].apply(lemmatize)
df

,Srt,Level
0,"[harvey, reading, put, gallo, away, racketeeri...",B2
1,"[cinderella, lovely, name, cinderella, sunset,...",B1
2,"[crowd, cheering, distance, lelgh, anne, momen...",B1
3,"[better, get, married, soon, starting, look, o...",A1-A2
4,"[wordskill, ff, edited, pop, music, playing, s...",B2
...,...,...
252,"[thought, liked, idea, realized, nothing, sass...",B2
253,"[america, irradiated, wasteiand, within, lies,...",A1-A2
254,"[begin, first, thing, need, know, mother, name...",B2
255,"[oh, come, land, faraway, place, caravan, came...",A1-A2


def lem_back(plurals):
    singles = [plural for plural in plurals]
    return (' '.join(singles))

df['Srt'] = df['Srt'].apply(lem_back)
df

- Стемминг (приведение слов у основе)

Стемминг – это своего рода нормализация слов. Нормализация – это метод, при котором набор слов в предложении преобразуется в последовательность, чтобы сократить время поиска. Слова, которые имеют то же значение, но имеют некоторые различия в зависимости от контекста или предложения, нормализуются.

Проведем нормализацию с помощью `PorterStemmer()`.

In [17]:
stemmer = PorterStemmer()

def stemming(plurals):
    singles = [stemmer.stem(plural) for plural in plurals]
    return (' '.join(singles))

df['Srt'] = df['Srt'].apply(stemming)
df

,Srt,Level
0,harvey read put gallo away racket realli talk ...,B2
1,cinderella love name cinderella sunset frame t...,B1
2,crowd cheer distanc lelgh ann moment oforderli...,B1
3,better get marri soon start look old dad say s...,A1-A2
4,wordskil ff edit pop music play string bind ha...,B2
...,...,...
252,thought like idea realiz noth sassi remark tho...,B2
253,america irradi wasteiand within lie citi outsi...,A1-A2
254,begin first thing need know mother name enola ...,B2
255,oh come land faraway place caravan camel roam ...,A1-A2


- Оценка релевантности слов с помощью частотно-обратной частоты термина в документе

In [18]:
docs = np.array(df['Srt'])
pipe = Pipeline([('count', CountVectorizer()),
                ('tfid', TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True))]).fit(docs)

In [19]:
tfidf = pd.DataFrame(pipe.transform(docs).toarray())
df_srt = pd.DataFrame(tfidf)
df_srt['Level'] = df['Level']
df_srt

,0,1,2,3,4,5,6,7,8,9,...,22550,22551,22552,22553,22554,22555,22556,22557,22558,Level
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B2
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.186133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A1-A2
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B2
253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A1-A2
254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B2
255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A1-A2


## 3. Обучение модели

In [20]:
X = df_srt.drop(['Level'], axis=1)
y = df_srt['Level']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12345)

In [21]:
learn_pool = Pool(
    X_train, 
    y_train
)

test_pool = Pool(
    X_test, 
    y_test
)

In [26]:
params = {
    'iterations': 1000,
    'eval_metric': 'TotalF1',
    'early_stopping_rounds': 200,
    'use_best_model': True,
    'verbose': 100,
    'loss_function': 'MultiClass'
}

In [ ]:
lr_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for i in lr_list:
    print()
    print()
    print('learning_rate:', i)
    print()
    
    model = CatBoostClassifier(**params, learning_rate=i)
    model.fit(learn_pool, eval_set=test_pool)
    
    predictions = model.predict(test_pool)
    print(metrics.classification_report(y_test, predictions, digits=3))



learning_rate: 0.1

0:	learn: 0.5463836	test: 0.4163998	best: 0.4163998 (0)	total: 1.3s	remaining: 21m 44s
100:	learn: 0.9895405	test: 0.4683126	best: 0.4999095 (1)	total: 1m 52s	remaining: 16m 37s
200:	learn: 0.9896237	test: 0.5263471	best: 0.5263471 (141)	total: 3m 47s	remaining: 15m 2s
300:	learn: 0.9895358	test: 0.5326309	best: 0.5445591 (241)	total: 5m 40s	remaining: 13m 11s
400:	learn: 0.9895405	test: 0.5445591	best: 0.5445591 (241)	total: 7m 31s	remaining: 11m 13s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.5445590994
bestIteration = 241

Shrink model to first 242 iterations.


C:\Users\yulia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yulia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yulia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       A1-A2      0.000     0.000     0.000        12
          B1      0.538     0.467     0.500        15
          B2      0.612     0.909     0.732        33
       C1-C2      1.000     0.600     0.750         5

    accuracy                          0.615        65
   macro avg      0.538     0.494     0.495        65
weighted avg      0.512     0.615     0.545        65



learning_rate: 0.2

0:	learn: 0.5463836	test: 0.4163998	best: 0.4163998 (0)	total: 1.28s	remaining: 21m 23s
100:	learn: 0.9896238	test: 0.5138600	best: 0.5376740 (85)	total: 1m 53s	remaining: 16m 48s
200:	learn: 0.9896238	test: 0.5441674	best: 0.5614058 (184)	total: 3m 45s	remaining: 14m 56s
300:	learn: 0.9895358	test: 0.5410919	best: 0.5614058 (184)	total: 5m 39s	remaining: 13m 8s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.5614058355
bestIteration = 184

Shrink model to first 185 iterations.
              precision    recall  f1-s

In [ ]:
lr_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for i in lr_list:
    params = {
    'learning_rate': i,
    'iterations': 1000,
    'eval_metric': 'TotalF1',
    'early_stopping_rounds': 200,
    'use_best_model': True,
    'verbose': 100,
    'loss_function': 'MultiClass'
    }
    
    print()
    print()
    print('learning_rate:', i)
    print()
    scores = cv(learn_pool,
                params,
                fold_count=3,
                )
    
    print(scores[scores['test-TotalF1-mean'] == scores['test-TotalF1-mean'].max()])

- Кросс-валидация и подбор гипер-параметра

In [ ]:
model = CatBoostClassifier(**params, learning_rate=0.8)
model.fit(learn_pool, eval_set=test_pool)
print()
predictions = model.predict(test_pool)
print()
print(metrics.classification_report(y_test, predictions, digits=3))

## 4. Предсказание уровня английского языка для загружаемого субтитра

In [ ]:
path = r'C:\Users\yulia\OneDrive\Документы\С рабочего стола\Проекты\Workshop_2\English_scores\Test\The_Social_Network_2010_BluRay_1080p_AVC_DTSMA_DL_Remux_TvR_en_non.srt'

In [ ]:
# Загрузка .srt
# Создать df с колокой Path, вставить туда путь к файлу
df_check = pd.DataFrame({'Path': [path]})
df_check['Srt'] = df_check.apply(add_column_srt, axis=1)

df_check['Srt'] = df_check.apply(replace_str, axis=1)

# Лемматизация
df_check['Srt'] = df_check['Srt'].apply(lemmatize)
df_check

df_check['Srt'] = df_check['Srt'].apply(lem_back)
df_check

# Стемминг
#df_check['Srt'] = df_check['Srt'].apply(stemming)
#df_check

docs_ch = np.array(df_check['Srt'])
#df_ch = pipe.transform(docs_ch)

df_ch = pd.DataFrame(pipe.transform(docs_ch).toarray())

pool = Pool(
    df_ch
)

# Предсказание
prediction = model.predict(pool)
print(prediction)